In [52]:
using CSV, JuMP, Gurobi, DataFrames;

In [53]:
GUROBI_ENV = Gurobi.Env();

Academic license - for non-commercial use only


# Reading Data

In [54]:
PLAYERS_DATA_PATH = "../data/optimization_input.csv";
OUTPUT_PATH = "../output/lineups.csv";
TEST_DATA_PATH = "../data/test_input.csv"

"../data/test_input.csv"

In [55]:
players = CSV.read(PLAYERS_DATA_PATH);

In [56]:
test_set = CSV.read(TEST_DATA_PATH);

In [57]:
players[:,:Game].pool

3-element Array{String,1}:
 "WAS@LAC"
 "UTA@TOR"
 "GS@ORL" 

# IP Formulation

In [58]:
function optimize_lineups(players; nb_lineups=1, max_overlap=6, fp_column="prediction", MAX_PG=2, MAX_SG=2, MAX_SF=2, MAX_PF=2, MAX_C=1, BUDGET=60000.0, save=true, output_path="../output/lineups.csv")
    # Columns names
    NAME = Symbole("name")
    TEAM = Symbole("team_key")
    POSITION = Symbol("Position")
    SALARY = Symbol("Salary")
    INJURY = Symbol("Injury Indicator")
    FP = Symbol(fp_column)

    # Reading inputs
    ## Fantasy points
    fp = players[:,FP];
    ## Positions
    PG = Int.(players[:,POSITION].=="PG")
    SG = Int.(players[:,POSITION].=="SG")
    SF = Int.(players[:,POSITION].=="SF")
    PF = Int.(players[:,POSITION].=="PF")
    C  = Int.(players[:,POSITION].=="C")
    ## Salary
    salary = players[:,SALARY];
    ## Injuries
    injury = 1 .- ismissing.(players[:,INJURY]) 
    o_injury = Int.(Missings.coalesce.(players[:,INJURY], 0).=="O")
    q_injury = Int.(Missings.coalesce.(players[:,INJURY], 0).=="Q")
    p_injury = Int.(Missings.coalesce.(players[:,INJURY], 0).=="P");

    ## Number of players
    nb_players = size(players)[1]

    # Model
    model = Model(solver=GurobiSolver(OutputFlag=0, GUROBI_ENV))

    # Variable
    @variable(model, z[i=1:nb_players], Bin)

    # Constrains without the overleap constraint
    @objective(model, Max, sum(fp.*z))
    @constraint(model, sum(salary.*z) <= BUDGET)
    @constraint(model, sum(PG.*z) == MAX_PG)
    @constraint(model, sum(SG.*z) == MAX_SG)
    @constraint(model, sum(SF.*z) == MAX_SF)
    @constraint(model, sum(PF.*z) == MAX_PF)
    @constraint(model, sum(C.*z) == MAX_C)
    @constraint(model, z .<= (1 .- injury))

    # Initialization (iteration=1)
    solve(model)
    x = Int.(getvalue(z))
    lineups = players[x.==1, [NAME, POSITION, TEAM , FP]]    
    names!(lineups, Symbol.([string("Name_",1), string("Position_",1), string("Team_",1), string("FP_",1)]))  

    # Rest of iterations
    @constraint(model, sum(x.*z) <= max_overlap)
    for i=2:nb_lineups
        solve(model)
        x = hcat(x,Int.(getvalue(z)))
        lineups_names = players[x[:,i].==1, NAME, POSITION, TEAM , FP]]
        names!(lineups_names, Symbol.([string("Name_",i), string("Position_",i), string("Team_",i), string("FP_",i)]))    
        lineups = hcat(lineups, lineups_names)
        @constraint(model, sum(x[:,i].*z) <= max_overlap)
    end
    if save
        CSV.write(output_path, lineups);
    end
    score_per_lineup = get_score_lineups(lineups)
    return lineups, score_per_lineup
end;
    

LoadError: syntax: extra token "]" after end of expression

# Annexe functions

In [59]:
function get_number_of_lineups(lineups; fp_column = "FP_")
    # Return the number of lineups using the columns fp_column_i
    colnames = names(lineups)
    nb_lineups = 0
    for col in colnames
        if startswith(string(col), fp_column)
            nb_lineups = nb_lineups+1
        end
    end
    nb_lineups
end;

In [116]:
function get_score_lineups(lineups; fp_column = "FP_")
    # Returns the scores of the lineups based on the columns 
    nb_lineups = get_number_of_lineups(lineups; fp_column = fp_column)
    return aggregate(lineups[:,Symbol.([string(fp_column,i) for i=1:nb_lineups])], sum)
end;

In [61]:
function add_true_scores(lineups, test_set; fp_column = "FP_", true_fp_column = "fp")
    # Adds the columns true_fp_column_i from the test_set to the lineups
    nb_lineups = get_number_of_lineups(lineups; fp_column = fp_column)
    for lineup=1:nb_lineups
        player_names = lineups[!,Symbol(string("Name_",lineup))]
        lineups[!,Symbol(string("True_FP_",lineup))] = filter(row -> row[:name] in player_names, test_set)[!,Symbol(true_fp_column)]
    end
    return order_lineups(lineups)
end;

In [62]:
function order_lineups(lineups)
    # Orders the column of lineups based on the lineup index
    nb_lineups = get_number_of_lineups(lineups)
    colnames = names(lineups)
    ordered_colnames = []
    for lineup=1:nb_lineups
        for col in colnames
            if endswith(string(col), string("_",lineup))
                push!(ordered_colnames, col)
            end
        end
    end
    return lineups[:,ordered_colnames]
end;

In [77]:
function get_lineup(lineups, lineup_idx)
    # Returns all the columns of the lineup with index lineup_idx
    all_colnames = names(lineups)
    lineup_colnames = []
    for col in all_colnames
        if endswith(string(col), string(lineup_idx))
            push!(lineup_colnames, col)
        end
    end
    return lineups[:,lineup_colnames]
end;

In [64]:
function get_best_lineup(lineups; fp_column = "FP_")
    # Returns the best lineup and its score based on the column fp_column_i
    nb_lineups = get_number_of_lineups(lineups)
    scores = get_score_lineups(lineups, fp_column=fp_column)
    best_lineup_idx = 1
    for lineup=2:nb_lineups
        if scores[1,lineup]>scores[1,best_lineup_idx]
            best_lineup_idx=lineup
        end
    end
    return get_lineup(lineups, best_lineup_idx), scores[:,[best_lineup_idx]]
end;

In [87]:
function test_lineups(lineups, test_set; pred_column = "prediction", true_column="fp")
    best_lineup, best_score = optimize_lineups(test_set, fp_column=true_column, nb_lineups=1)
    lineup_with_true_scores = add_true_scores(lineups, test_set; fp_column = "FP_", true_fp_column = true_column)
    best_lineup_with_true_scores, best_lineup_true_score = get_best_lineup(lineup_with_true_scores; fp_column = "True_FP_")
    captured_score = (best_lineup_true_score[1,1]/best_score[1,1])*100
    return best_lineup_with_true_scores, best_lineup, captured_score
end;

# Testing Functions

In [69]:
lineups, lineups_scores =  optimize_lineups(players; nb_lineups=10, max_overlap=6, fp_column="prediction", save=true, output_path="../output/lineups.csv");

In [70]:
lineups

,Name_1,Position_1,Team_1,FP_1,Name_2,Position_2,Team_2
,String,String,String,Float64,String,String,String
1,Bradley Beal,SG,WAS,53.4317,Bradley Beal,SG,WAS
2,Paul George,SF,LAC,54.6104,Paul George,SF,LAC
3,Fred VanVleet,PG,TOR,43.2081,Donovan Mitchell,SG,UTA
4,Bojan Bogdanovic,SF,UTA,39.0917,Bojan Bogdanovic,SF,UTA
5,Evan Fournier,SG,ORL,39.2203,Mike Conley,PG,UTA
6,Aaron Gordon,PF,ORL,34.4912,Eric Paschall,PF,GS
7,D.J. Augustin,PG,ORL,28.7639,D.J. Augustin,PG,ORL
8,Mo Bamba,C,ORL,25.7915,Mo Bamba,C,ORL
9,Omari Spellman,PF,GS,27.3356,Omari Spellman,PF,GS


In [71]:
lineups_scores

,FP_1_sum,FP_2_sum,FP_3_sum,FP_4_sum,FP_5_sum,FP_6_sum,FP_7_sum,FP_8_sum
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,345.944,343.864,343.627,343.493,343.425,343.335,343.281,343.141


In [72]:
get_number_of_lineups(lineups; fp_column = "FP_")

10

In [117]:
get_score_lineups(lineups; fp_column = "FP_")

,FP_1_sum,FP_2_sum,FP_3_sum,FP_4_sum,FP_5_sum,FP_6_sum,FP_7_sum,FP_8_sum
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,345.944,343.864,343.627,343.493,343.425,343.335,343.281,343.141


In [75]:
lineup_with_true_scores = add_true_scores(lineups, test_set; fp_column = "FP_", true_fp_column = "fp")

,Name_1,Position_1,Team_1,FP_1,True_FP_1,Name_2,Position_2
,String,String,String,Float64,Float64,String,String
1,Bradley Beal,SG,WAS,53.4317,58.7749,Bradley Beal,SG
2,Paul George,SF,LAC,54.6104,60.0715,Paul George,SF
3,Fred VanVleet,PG,TOR,43.2081,47.5289,Donovan Mitchell,SG
4,Bojan Bogdanovic,SF,UTA,39.0917,35.1826,Bojan Bogdanovic,SF
5,Evan Fournier,SG,ORL,39.2203,35.2983,Mike Conley,PG
6,Aaron Gordon,PF,ORL,34.4912,37.9403,Eric Paschall,PF
7,D.J. Augustin,PG,ORL,28.7639,25.8875,D.J. Augustin,PG
8,Mo Bamba,C,ORL,25.7915,28.3707,Mo Bamba,C
9,Omari Spellman,PF,GS,27.3356,24.602,Omari Spellman,PF


In [78]:
get_lineup(lineup_with_true_scores, 2)

,Name_2,Position_2,Team_2,FP_2,True_FP_2
,String,String,String,Float64,Float64
1,Bradley Beal,SG,WAS,53.4317,58.7749
2,Paul George,SF,LAC,54.6104,60.0715
3,Donovan Mitchell,SG,UTA,46.0897,50.6986
4,Bojan Bogdanovic,SF,UTA,39.0917,35.1826
5,Mike Conley,PG,UTA,35.6256,39.1882
6,Eric Paschall,PF,GS,33.1242,29.8117
7,D.J. Augustin,PG,ORL,28.7639,25.8875
8,Mo Bamba,C,ORL,25.7915,28.3707
9,Omari Spellman,PF,GS,27.3356,24.602


In [83]:
# Returns the best lineup based on the predictions
get_best_lineup(lineup_with_true_scores; fp_column = "FP_")

(9×5 DataFrame
│ Row │ Name_1           │ Position_1 │ Team_1 │ FP_1    │ True_FP_1 │
│     │ String           │ String     │ String │ Float64 │ Float64   │
├─────┼──────────────────┼────────────┼────────┼─────────┼───────────┤
│ 1   │ Bradley Beal     │ SG         │ WAS    │ 53.4317 │ 58.7749   │
│ 2   │ Paul George      │ SF         │ LAC    │ 54.6104 │ 60.0715   │
│ 3   │ Fred VanVleet    │ PG         │ TOR    │ 43.2081 │ 47.5289   │
│ 4   │ Bojan Bogdanovic │ SF         │ UTA    │ 39.0917 │ 35.1826   │
│ 5   │ Evan Fournier    │ SG         │ ORL    │ 39.2203 │ 35.2983   │
│ 6   │ Aaron Gordon     │ PF         │ ORL    │ 34.4912 │ 37.9403   │
│ 7   │ D.J. Augustin    │ PG         │ ORL    │ 28.7639 │ 25.8875   │
│ 8   │ Mo Bamba         │ C          │ ORL    │ 25.7915 │ 28.3707   │
│ 9   │ Omari Spellman   │ PF         │ GS     │ 27.3356 │ 24.602    │, 1×1 DataFrame
│ Row │ FP_1_sum │
│     │ Float64  │
├─────┼──────────┤
│ 1   │ 345.944  │)

In [85]:
# Returns the best lineup based on the true FP scores
get_best_lineup(lineup_with_true_scores; fp_column = "True_FP_")

(9×5 DataFrame
│ Row │ Name_4         │ Position_4 │ Team_4 │ FP_4    │ True_FP_4 │
│     │ String         │ String     │ String │ Float64 │ Float64   │
├─────┼────────────────┼────────────┼────────┼─────────┼───────────┤
│ 1   │ Bradley Beal   │ SG         │ WAS    │ 53.4317 │ 58.7749   │
│ 2   │ Paul George    │ SF         │ LAC    │ 54.6104 │ 60.0715   │
│ 3   │ Fred VanVleet  │ PG         │ TOR    │ 43.2081 │ 47.5289   │
│ 4   │ Evan Fournier  │ SG         │ ORL    │ 39.2203 │ 35.2983   │
│ 5   │ Aaron Gordon   │ PF         │ ORL    │ 34.4912 │ 37.9403   │
│ 6   │ Eric Paschall  │ PF         │ GS     │ 33.1242 │ 29.8117   │
│ 7   │ Marc Gasol     │ C          │ TOR    │ 28.5988 │ 31.4587   │
│ 8   │ Glenn Robinson │ SF         │ GS     │ 28.0443 │ 30.8487   │
│ 9   │ D.J. Augustin  │ PG         │ ORL    │ 28.7639 │ 25.8875   │, 1×1 DataFrame
│ Row │ True_FP_4_sum │
│     │ Float64       │
├─────┼───────────────┤
│ 1   │ 357.62        │)

In [89]:
best_lineup_with_true_scores, best_lineup, captured_score = test_lineups(lineups, test_set; pred_column = "prediction", true_column="fp");

In [90]:
best_lineup_with_true_scores

,Name_4,Position_4,Team_4,FP_4,True_FP_4
,String,String,String,Float64,Float64
1,Bradley Beal,SG,WAS,53.4317,58.7749
2,Paul George,SF,LAC,54.6104,60.0715
3,Fred VanVleet,PG,TOR,43.2081,47.5289
4,Evan Fournier,SG,ORL,39.2203,35.2983
5,Aaron Gordon,PF,ORL,34.4912,37.9403
6,Eric Paschall,PF,GS,33.1242,29.8117
7,Marc Gasol,C,TOR,28.5988,31.4587
8,Glenn Robinson,SF,GS,28.0443,30.8487
9,D.J. Augustin,PG,ORL,28.7639,25.8875


In [119]:
sum(best_lineup_with_true_scores[!,:True_FP_4])

357.62049505000005

In [120]:
best_lineup

,Name_1,Position_1,Team_1,FP_1
,String,String,String,Float64
1,Bradley Beal,SG,WAS,58.7749
2,Paul George,SF,LAC,60.0715
3,Fred VanVleet,PG,TOR,47.5289
4,Draymond Green,PF,GS,40.2631
5,Mike Conley,PG,UTA,39.1882
6,Aaron Gordon,PF,ORL,37.9403
7,Glenn Robinson,SF,GS,30.8487
8,Mo Bamba,C,ORL,28.3707
9,Jordan McRae,SG,WAS,29.662


In [122]:
sum(best_lineup[!,:FP_1])

372.64820273

In [123]:
captured_score

95.96732049962732